In [56]:
import random

import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

import tengp
import symreg
from gpbenchmarks import get_data

from experiment_settings import pagie_funset, keijzer_funset, vlad_funset

def inactive(x, y):
    return x
def pdivide(x, y):
    return np.divide(x, y, out=np.copy(x), where=x!=0)

def plog(x, y):
    return np.log(x, out=np.copy(x), where=x>0)

def psin(x, y):
    return np.sin(x)

def pcos(x, y):
    return np.cos(x)

def pow(x, y):
    return np.power(x, y)

def pow2(x, y):
    return x**2

def pow3(x, y):
    return x**3

def ptan(x, y):
    return np.tan(x)

def ptanh(x, y):
    return np.tanh(x)

def psqrt(x, y):
    return  np.sqrt(x)

def pexp(x, y):
    return np.exp(x)

def pow_minus1(x, y):
    return x**(-1)
nguyen7_funset = tengp.FunctionSet()
nguyen7_funset.add(inactive, 2)
nguyen7_funset.add(np.add, 2)
nguyen7_funset.add(inactive, 2)
nguyen7_funset.add(np.subtract, 2)
nguyen7_funset.add(inactive, 2)
nguyen7_funset.add(np.multiply, 2)
nguyen7_funset.add(inactive, 2)
nguyen7_funset.add(pdivide, 2)
nguyen7_funset.add(inactive, 2)
nguyen7_funset.add(plog, 2)
nguyen7_funset.add(inactive, 2)
nguyen7_funset.add(psin, 2)
nguyen7_funset.add(inactive, 2)
nguyen7_funset.add(pcos, 2)
nguyen7_funset.add(inactive, 2)

pagie_funset = tengp.FunctionSet()
nguyen7_funset.add(inactive, 2)
pagie_funset.add(np.add, 2)
nguyen7_funset.add(inactive, 2)
pagie_funset.add(np.subtract, 2)
nguyen7_funset.add(inactive, 2)
pagie_funset.add(np.multiply, 2)
nguyen7_funset.add(inactive, 2)
pagie_funset.add(pdivide, 2)
nguyen7_funset.add(inactive, 2)
pagie_funset.add(np.power, 2)
nguyen7_funset.add(inactive, 2)
pagie_funset.add(psqrt, 2)
nguyen7_funset.add(inactive, 2)
pagie_funset.add(plog, 2)
nguyen7_funset.add(inactive, 2)
pagie_funset.add(pow_minus1, 2)
nguyen7_funset.add(inactive, 2)


#     function set: +, -, *, /, sin, cos, tan, tanh, sqrt, exp, log, **2, **3
korns12_funset = tengp.FunctionSet()
korns12_funset.add(inactive, 2)
korns12_funset.add(np.add, 2)
korns12_funset.add(inactive, 2)
korns12_funset.add(np.subtract, 2)
korns12_funset.add(inactive, 2)
korns12_funset.add(np.multiply, 2)
korns12_funset.add(inactive, 2)
korns12_funset.add(pdivide, 2)
korns12_funset.add(inactive, 2)
korns12_funset.add(psin, 2)
korns12_funset.add(inactive, 2)
korns12_funset.add(pcos, 2)
korns12_funset.add(inactive, 2)
korns12_funset.add(ptan, 2)
korns12_funset.add(inactive, 2)
korns12_funset.add(ptanh, 2)
korns12_funset.add(inactive, 2)
korns12_funset.add(psqrt, 2)
korns12_funset.add(inactive, 2)
korns12_funset.add(pexp, 2)
korns12_funset.add(inactive, 2)
korns12_funset.add(plog, 2)
korns12_funset.add(inactive, 2)
korns12_funset.add(pow2, 2)
korns12_funset.add(inactive, 2)
korns12_funset.add(pow3, 2)
korns12_funset.add(inactive, 2)


In [2]:
import pygmo as pg

In [26]:
vectors = []
n_actives = []

class cost_function:
    def __init__(self, X, Y, params, bounds):
        self.params = params
        self.bounds = bounds
        self.X = X
        self.Y = Y
    
    def fitness(self, x):
        #vectors.append(x)
        
        individual = tengp.individual.NPIndividual(list(x), self.bounds, self.params)
        pred = individual.transform(self.X)
        
        #n_actives.append(len(individual.active_nodes))
        
        try:
            return [mean_squared_error(pred, self.Y)]
        except ValueError:
            return [10000000000]
        
    def get_bounds(self):
        return self.bounds

In [57]:
kw_params = {'real_valued': True, 'max_back': 30}

params_nguyen7 = tengp.Parameters(2, 1, 1, 50, nguyen7_funset, **kw_params)
params_pagie1  = tengp.Parameters(2, 1, 1, 50, pagie_funset, **kw_params)
params_keijzer6 = tengp.Parameters(2, 1, 1, 50, keijzer_funset, **kw_params)
params_korns = tengp.Parameters(6, 1, 1, 50, korns12_funset, **kw_params)
params_vlad = tengp.Parameters(6, 1, 1, 50, vlad_funset, **kw_params)

In [58]:
import random 

data = [
    ('nguyen7', symreg.get_benchmark_nguyen7(random, None), params_nguyen7),
    ('pagie1', symreg.get_benchmark_pagie1(random, None), params_pagie1),
    ('keijzer6', symreg.get_benchmark_keijzer(random, 6), params_keijzer6),
    ('korns12', symreg.get_benchmark_korns(random, 12), params_korns),
    ('vladislasleva4', symreg.get_benchmark_vladislasleva4(random, None), params_vlad)
]

In [59]:
%%time
results = []
champions = []


for name, (x_train, y_train, x_test, y_test), params in data[1:2]:
    print(name)
    bounds = tengp.individual.IndividualBuilder(params).create().bounds[:]
    
    prob = pg.problem(cost_function(np.c_[x_train, np.ones(len(x_train))], y_train, params, bounds))
    
    problem_results = []
    pg.set_global_rng_seed(42)
    for i in range(1):
        print(i, end=',')
        algo = pg.algorithm(pg.pso(gen=200))
        algo.set_verbosity(1)
        pop = pg.population(prob, 50)
        pop = algo.evolve(pop)
        print(pop.champion_f[0])
        problem_results.append(pop.champion_f[0])
        champions.append(pop.champion_x)
    results.append(problem_results)

pagie1
0,

/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:102: RuntimeWarning: invalid value encountered in power
  l = (1-a)*f_l(L, U)
/home/jarino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in sqrt
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:102: RuntimeWarning: overflow encountered in multiply
  l = (1-a)*f_l(L, U)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:102: RuntimeWarning: overflow encountered in power
  l = (1-a)*f_l(L, U)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:103: RuntimeWarning: invalid value encountered in power
  u = a*f_u(L, U)
/home/jarino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide
/home/jarino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in greater
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:103: RuntimeWarning: ov

0.13415882357179362
CPU times: user 13.3 s, sys: 78.1 ms, total: 13.3 s
Wall time: 14.3 s


In [46]:
results

[[0.0006599427389086961],
 [0.8908309892604264],
 [0.025333844251682468],
 [1.1158593319483283],
 [0.03706425662868222]]

In [ ]:
def plot_results(name, outs):
    print(name)
    costs = np.array(outs)
    print('min:', np.min(costs))
    print('mean:', np.mean(costs))
    print('median:', np.median(costs))
    print('variance:',np.var(costs))
    plt.figure()
    plt.title(name)
    sns.distplot(costs, kde=False);
    

In [ ]:
for d, result in zip(data, results):
    plot_results(d[0], result)